In [1]:
from circuit import gen_random_circuits
from upstream import RandomwalkModel
from collections import defaultdict

import math
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend
# from utils.backend import default_basis_single_gates, default_basis_two_gates


import pennylane as qml
import numpy as np
from downstream.synthesis.pennylane_op import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op import layer_circuit_to_matrix
from downstream.synthesis import matrix_distance_squared

grid_num = 2
topology = gen_grid_topology(grid_num) # 3x3 9 qubits
neigh_info = get_grid_neighbor_info(grid_num, 1)


n_qubits = grid_num ** 2
backend = Backend(n_qubits=n_qubits, topology=topology, neighbor_info=neigh_info, basis_single_gates = ['u'], 
                  basis_two_gates = ['cz'], divide = False, decoupling=False)



from downstream.synthesis import SynthesisModel
from scipy.stats import unitary_group

In [2]:
import cloudpickle as pickle
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend
from circuit import gen_random_circuits
from upstream import RandomwalkModel
from downstream.synthesis.pennylane_op import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op import layer_circuit_to_matrix
from downstream.synthesis.neural_network import NeuralNetworkModel
from sklearn.neural_network import MLPRegressor as DNN

from jax import numpy as jnp
from jax import vmap
import jax
import optax
from jax.config import config
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from scipy.stats import unitary_group
import random
import copy
import ray
from sklearn.model_selection import train_test_split


In [3]:
def transformU(U):
    '''转成神经网络里面的格式'''
    U = U.reshape(U.size)
    U = jnp.concatenate([U.imag, U.real])
    U = jnp.array(U, dtype = jnp.float64)
    return U

In [6]:
max_gate = 200 #4**n_qubits

'''生成用于测试的模板电路'''
circuits = gen_random_circuits(min_gate = 100, max_gate = max_gate, gate_num_step = max_gate//50, n_circuits = 1, two_qubit_gate_probs=[4, 8], backend = backend, reverse = False, optimize = True)

upstream_model = RandomwalkModel(1, 20, backend = backend, travel_directions=('parallel', 'next'))
upstream_model.train(circuits, multi_process = True)

def random_params(layer2gates):
    layer2gates = copy.deepcopy(layer2gates)
    for layer in layer2gates:
        for gate in layer:
            gate['params'] = [
                random.random() * 2 * jnp.pi
                for param in gate['params']
            ]
    return layer2gates

@ray.remote
def _gen_data(circuit_info, index, n_qubits): 
    if index % 1000 == 0:
        print('finished rate', index)
    gate_vecs, Us = [], []
    layer2gates = circuit_info['layer2gates']
    # print(layer2gates)
    for layer_index, layer_gates in enumerate(layer2gates):
        layer_gates = [
            gate
            for gate in layer_gates
            if len(gate['qubits']) == 1
        ]
        if len(layer_gates) == 0:
            continue
        for _ in range(1):
            target_gate_index = random.randint(0, len(layer_gates)-1)
            target_gate_index = layer_gates[target_gate_index]['id']
            target_gate = circuit_info['gates'][target_gate_index]

            gate_vec = circuit_info['vecs'][target_gate_index]
        
            _layer2gates = random_params(layer2gates[layer_index:])
            U = layer_circuit_to_matrix(_layer2gates, n_qubits)  #TODO: vmap
            U = transformU(U)
            
            # 似乎根本没有出现过
            # _dists = vmap(matrix_distance_squared, in_axes=(None, 0))(U, jnp.array(U, dtype=jnp.complex128))
            # if jnp.any(_dists < 0.1):
            #     continue
            
            Us.append(U)
            gate_vecs.append(np.concatenate([jnp.array(target_gate['qubits']), gate_vec], axis=0)  )
            
    return gate_vecs, Us

print('start gen Us -> gate_vectos')
futures = []
for index, circuit_info in enumerate(circuits):
    # upstream_model.vectorize(circuit_info)
    future = _gen_data.remote(circuit_info, index/len(circuits), n_qubits)
    # future = _gen_data(circuit_info, n_qubits)
    futures.append(future)
    
gate_vecs, Us = [], []
for future in futures:
    if not isinstance(future, tuple):
        future = ray.get(future)
    gate_vecs += future[0]
    Us += future[1]

gate_vecs = jnp.array(gate_vecs, dtype=jnp.float64)
Us = jnp.array(Us, dtype=jnp.float64)
# Us, Us_test, gate_vecs, gate_vecs_test = train_test_split(Us, gate_vecs, test_size=.1)

len(Us), len(gate_vecs), #len(Us_test), len(gate_vecs_test)

KeyboardInterrupt: 

In [ ]:
neural_model = DNN((4**n_qubits, 2**n_qubits, 2**n_qubits, 4*n_qubits, n_qubits), verbose = True, n_iter_no_change=20)
neural_model.fit(Us, gate_vecs)
# neural_model.score(transform(Us_test), gate_vecs_test)

Iteration 1, loss = 0.39118028
Iteration 2, loss = 0.11691558
Iteration 3, loss = 0.08643137
Iteration 4, loss = 0.08515224
Iteration 5, loss = 0.08440917
Iteration 6, loss = 0.08388689
Iteration 7, loss = 0.08348472
Iteration 8, loss = 0.08308084
Iteration 9, loss = 0.08269488
Iteration 10, loss = 0.08233844
Iteration 11, loss = 0.08198173
Iteration 12, loss = 0.08164873
Iteration 13, loss = 0.08128136
Iteration 14, loss = 0.08091604
Iteration 15, loss = 0.08062071
Iteration 16, loss = 0.08021890
Iteration 17, loss = 0.07988895
Iteration 18, loss = 0.07854912
Iteration 19, loss = 0.07585225
Iteration 20, loss = 0.07434398
Iteration 21, loss = 0.07323662
Iteration 22, loss = 0.07226610
Iteration 23, loss = 0.07142047
Iteration 24, loss = 0.07075557
Iteration 25, loss = 0.06997065
Iteration 26, loss = 0.06946654
Iteration 27, loss = 0.06890068
Iteration 28, loss = 0.06838322
Iteration 29, loss = 0.06789403
Iteration 30, loss = 0.06739235
Iteration 31, loss = 0.06706601
Iteration 32, los

-0.1130812922039489

In [ ]:
from sklearn.utils import shuffle
def step_function(x):
    return jnp.where(x>0, 1, 0)

def final_mapping(mapping, x):
    return step_function(mapping['W'] * x + mapping['B'])

def final_mapping_cost(mapping, x, y):
    predictions = final_mapping(mapping, x)
    return optax.l2_loss(predictions, y).sum()

mapping  = {
    'W': jax.random.normal(shape=(upstream_model.max_table_size,), key=jax.random.PRNGKey((random.randint(0, 10))), dtype=jnp.float64),
    'B': jnp.zeros(shape=(6,), dtype=jnp.float64)
},
opt = optax.adam(learning_rate=1e-2)
opt_state = opt.init(mapping)

# X, X_test, Y, Y_test = train_test_split(, Y, test_size=0.2)
X = neural_model.predict(Us)
Y = gate_vecs

batch_size = 5

best_loss = 1e10
best_mapping = mapping
for epoch in range(100):
    total_loss = 0
    X, Y = shuffle(X, Y)
    for start in range(0, len(X), batch_size):
        _X, _Y = X[start: start+batch_size], Y[start: start+batch_size]
        
        loss_value, gradient = jax.value_and_grad(final_mapping_cost)(mapping, _X, _Y)
        updates, opt_state = opt.update(gradient, opt_state, mapping)
        mapping = optax.apply_updates(mapping, updates) 
        total_loss += loss_value    
        
    if epoch%10 ==0:
        test_loss = final_mapping_cost(mapping, X_test, Y_test)
        print('epoch  %i\t| train loss = %.5f\t|  test loss = %.5f' % (epoch, total_loss, test_loss))
    
    if test_loss < best_loss:
        best_loss = test_loss
        best_mapping = mapping

Array([[-0.2330098 ,  0.10417514, -0.08606198, ..., -0.19171251,
         0.029759  ,  0.3081916 ],
       [ 0.33108756, -0.08831863,  0.19650535, ..., -0.10447828,
         0.33772954, -0.21738027],
       [ 0.23013577, -0.28185242,  0.21159315, ..., -0.31406161,
         0.18397439,  0.18440095],
       ...,
       [ 0.0314794 ,  0.04148141,  0.0392296 , ...,  0.04160149,
         0.03138558,  0.02243754],
       [ 0.74332678, -0.43117344,  0.        , ...,  0.        ,
        -0.24508424,  0.34086859],
       [ 0.75493485,  0.64480484,  0.        , ...,  0.        ,
        -0.05997806,  0.10345414]], dtype=float64)

In [ ]:
# neural_model = NeuralNetworkModel([4**n_qubits, 2**n_qubits, 2**n_qubits, n_qubits, upstream_model.max_table_size + 1])
# neural_model.fit(Us, gate_vecs)